# Processing SpikeGLX binary files
When you record with the SpikeGLX system it saves the data to binary files (\*.bin) with accompanying meta data (\*.meta). This includes both the neural data, which has an \*.imecX.\* string in its file names, and task/behavior data recorded with the NiDAQ board and has an \*.nidq.\* string. Those files ending in *.ap.bin can be subjected to spike sorting to extract the firing of individual neurons. Consult the tutorial 'RunSpikeSorting.ipynb' to learn how to do that.

Once spike sorting is completed, two additional processing steps should be performed: extraction of the local field potential and time alignment. This tutorial covers how to do these steps.

### Identify the files to process
For both steps we have to find the files to process. To do that, we will search the directory containing all the SpikeGLX data from a given recording session.

In [14]:
from mimo_pack.util.files import find_all_matching_files
import os

# find all the files in a directory and its subdirectories
exp_dir = "D:\\SortingTemp\\24-05-31_DH23_A1-IC_g0" # set this to the directory for the recording session
ap_files = find_all_matching_files(exp_dir, r't0\.imec([0-9]+)\.ap\.bin')
nidq_file = find_all_matching_files(exp_dir, r't0\.nidq\.bin')[0] # there should be only one nidq file

print("ap files:")
for f in ap_files:
    print(f)

print("nidq file:")
print(nidq_file)

ap files:
D:\SortingTemp\24-05-31_DH23_A1-IC_g0\24-05-31_DH23_A1-IC_g0_imec0\24-05-31_DH23_A1-IC_g0_t0.imec0.ap.bin
D:\SortingTemp\24-05-31_DH23_A1-IC_g0\24-05-31_DH23_A1-IC_g0_imec1\24-05-31_DH23_A1-IC_g0_t0.imec1.ap.bin
nidq file:
D:\SortingTemp\24-05-31_DH23_A1-IC_g0\24-05-31_DH23_A1-IC_g0_t0.nidq.bin


### Extraction of LFP
To extract the LFP signals, you can use functions found in `mimo_pack.preprocess.lfp`. These take an *.ap.bin binary file, low pass filter it to remove high frequency signals and reduce aliasing, downsample the signal, and then save it to a new binary file with the same base name but ending in *.lfp.bin. The function `make_lfp_file_spikeglx` by default low pass filters at 500 Hz and resamples at 1000 Hz. Following creation of the LFP binary file, a dclut meta file can be created using the *.meta file associated with the original *.ap.bin binary file.

In [2]:
from mimo_pack.preprocess.lfp import make_lfp_file_spikeglx, dclut_from_meta_lfp

lfp_files = []
lfp_dclut_files = []
# make a LFP file from the imec files
for ap_file in ap_files:
    print('Processing {}'.format(os.path.basename(ap_file)))
    lfp_files.append(make_lfp_file_spikeglx(ap_file, verbose=True))
    lfp_dclut_files.append(dclut_from_meta_lfp(lfp_files[-1]))

Processing D:\SortingTemp\24-05-31_DH23_A1-IC_g0\24-05-31_DH23_A1-IC_g0_imec0\24-05-31_DH23_A1-IC_g0_t0.imec0.ap.bin


100%|██████████| 451/451 [7:37:36<00:00, 60.88s/it]  


Processing D:\SortingTemp\24-05-31_DH23_A1-IC_g0\24-05-31_DH23_A1-IC_g0_imec1\24-05-31_DH23_A1-IC_g0_t0.imec1.ap.bin


100%|██████████| 451/451 [7:26:13<00:00, 59.36s/it]  


### Time alignment
Next we want to ensure that a common temporal reference frame is shared across all our files. For that we choose the first imec recording file (*.imec0.ap.bin) to be our reference and all other files are keyed off of that one. Each imec file will have a synchronization signal on channel 384. For the nidq file, the synchronization signal will be on whichever channel it had been plugged into on the NiDAQ card.

The alignment is stored using dclut meta files. When we created our LFP files these were made, but they have not yet been created for the *.ap.bin and *.nidq.bin files. So first we will create those.

In [15]:
# Create dclut files for ap and nidq files
from mimo_pack.fileio.spikeglx import dclut_from_meta

ap_dclut_files = [dclut_from_meta(f) for f in ap_files]
nidq_dclut_file = dclut_from_meta(nidq_file)


Now that all our files have associated dclut files, we can align them. The function `align_sync_dclut` loads the synchonization signal from two binary files (a *target* and a *reference*) with associated dclut meta data, identifies where those synchronization signals align, creates a table of the indices in the *target* file and their corresponding time stamps in the *reference* file, and returns a new dclut file for the *target* file with a new time scale that uses the alignment table. `align_sync_dclut` has five required inputs:
1. path_t : a string of the file path to the target's dclut file
2. path_r : a string of the file path to the reference's dclut file
3. sync_t : a dictionary passed to the `.points` method of a dclut object that specifies the synchronization channel for the target, e.g. `{'channel': [384]}`
4. sync_r : a dictionary passed to the `.points` method of a dclut object that specifies the synchronization channel for the reference.
5. sync_scale_name : a string with the name of the scale that corresponds time in both dclut files, e.g. `'time'`


In [18]:
from mimo_pack.preprocess.time import align_sync_dclut

# Align the AP files to the first one
ap_r = ap_dclut_files[0] 
sync_pt = {'channel': [384]}
sync_scale_name = 'time'
for ap_t in range(1, len(ap_dclut_files)):
    file_t = ap_dclut_files[ap_t]
    print('Aligning {} to {}'.format(os.path.basename(file_t), os.path.basename(ap_r)))
    align_sync_dclut(file_t, ap_r, sync_pt, sync_pt, sync_scale_name, verbose=True)

# Align the LFP files to the first AP file
for lfp_t in range(len(lfp_dclut_files)):
    file_t = lfp_dclut_files[lfp_t]
    print('Aligning {} to {}'.format(os.path.basename(file_t), os.path.basename(ap_r)))
    align_sync_dclut(file_t, ap_r, sync_pt, sync_pt, sync_scale_name, verbose=True)

# Align the NIDQ file to the first AP file
sync_nidq_pt = {'channel': [5]}
print('Aligning {} to {}'.format(os.path.basename(nidq_dclut_file), os.path.basename(ap_r)))
align_sync_dclut(nidq_dclut_file, ap_r, sync_nidq_pt, sync_pt, sync_scale_name, verbose=True)

Aligning 24-05-31_DH23_A1-IC_g0_t0.imec1.ap_dclut.json to 24-05-31_DH23_A1-IC_g0_t0.imec0.ap_dclut.json
Reading target file


100%|██████████| 451/451 [25:33<00:00,  3.40s/it]


Reading reference file


100%|██████████| 451/451 [25:36<00:00,  3.41s/it]


Aligning sync signals


...

100%|██████████| 451/451 [25:26<00:00,  3.38s/it]


Aligning sync signals


100%|██████████| 33688/33688 [00:03<00:00, 8542.40it/s]


> c:\programdata\anaconda3\envs\mimo_env\lib\site-packages\dclut\core.py(432)save()
    430 
    431         # save the dclut file
--> 432         with open(path, 'w') as f:
    433             json.dump(dcl, f)
    434 



'D:\\SortingTemp\\24-05-31_DH23_A1-IC_g0\\24-05-31_DH23_A1-IC_g0_t0.nidq_dclut.json'